In [ ]:
from iduedu import get_4326_boundary
from iduedu import get_single_public_transport_graph

# poly = get_4326_boundary(osm_id=1114252)

g = get_single_public_transport_graph('subway', osm_id=1114252)

In [ ]:
from iduedu import get_4326_boundary
from iduedu import get_single_public_transport_graph, get_intermodal_graph, get_all_public_transport_graph

poly = get_4326_boundary(osm_id=337422)
intermodal = get_intermodal_graph(territory=poly)

In [ ]:
from iduedu import graph_to_gdf

g_gdf = graph_to_gdf(g,nodes=True, restore_edge_geom=True)
g_gdf.explore(column='type',tiles='cartodb positron')

In [ ]:
poly = get_4326_boundary(osm_id=1114252)

In [ ]:
g_gdf.explore(column='type',tiles='cartodb positron')

In [ ]:
g_gdf['type'].unique()

In [ ]:
import networkx as nx

nx.write_gml(intermodal, 'spb_new_iduedu.gml', stringizer=lambda x: str(x))

In [ ]:
from shapely import Polygon
import pandas as pd
from iduedu.modules.overpass_downloaders import _poly_to_overpass, _overpass_request
from iduedu import config
from iduedu import get_4326_boundary

poly = get_4326_boundary(osm_id=175905)


def get_subway_entrances(polygon: Polygon) -> pd.DataFrame:
    polygon_coords = _poly_to_overpass(polygon)
    print(polygon_coords)
    overpass_query = \
        f"""
        [out:json][timeout:500];
        rel(poly:"{polygon_coords}")["route"="subway"]->.routes;
        node(r.routes)-> .route_nodes;
        rel(bn.route_nodes)->.stop_areas;

        rel(br.stop_areas)["public_transport"="stop_area_group"]["type"="public_transport"]->.groups;

        nwr(r.stop_areas)["public_transport"="station"]->.stations;

        .stop_areas     out geom qt;
        .groups out body qt;
        .stations out tags  qt;
        """

    resp = _overpass_request(
        method="POST",
        overpass_url=config.overpass_url,
        data={"data": overpass_query},
    )
    json_result = resp.json()["elements"]
    if len(json_result) == 0:
        return pd.DataFrame()

    for e in json_result:
        tags = e.get("tags") or {}
        etype = e.get("type")

        e["is_stop_area"] = (etype == "relation" and tags.get("public_transport") == "stop_area")
        e["is_stop_area_group"] = (
                etype == "relation"
                and tags.get("public_transport") == "stop_area_group"
                and tags.get("type") == "public_transport"
        )
        e["is_station"] = (tags.get("public_transport") == "station")

    data = pd.DataFrame(json_result)
    data["transport_type"] = "subway"
    return data


data = get_subway_entrances(poly)

In [ ]:
data

In [ ]:
stop_areas = data[data["is_stop_area"]]
stop_areas_group = data[data["is_stop_area_group"]]
stations_data = data[data["is_station"]]

In [ ]:
from collections import defaultdict
from itertools import chain
from shapely import LineString
from itertools import combinations

graph_nodes = []
graph_edges = []

station_parent_ref = {}


def _collect_by_roles(members, parent_id):
    """Собирает членов по ролям с учётом распространённых синонимов."""
    roles = defaultdict(list)
    for m in (members or []):
        role = (m.get("role") or "").lower()
        roles[role].append(m)

    # базовые наборы
    stations = roles.get("station", [])
    for s in stations:
        station_parent_ref[parent_id] = s['ref']
    stops = roles.get("stop", [])  # stop_position часто идёт как role=stop
    platforms = roles.get("platform", [])

    # входы/выходы и варианты подписей
    entrances = (
            roles.get("entrance", [])
            + roles.get("subway_entrance", [])
            + roles.get("entrance_yes", [])
            + roles.get("entrance_main", [])
    )
    entry_only = (
            roles.get("entry_only", [])
            + roles.get("entrance_entry_only", [])
            + roles.get("entry", [])
    )
    exit_only = (
            roles.get("exit_only", [])
            + roles.get("entrance_exit_only", [])
            + roles.get("exit", [])
    )

    return stations, stops, platforms, entrances, entry_only, exit_only


def add_node(ref_id, x, y, node_type):
    graph_nodes.append(
        {
            "ref_id": int(ref_id),
            "point": (x, y),
            "type": node_type,
        }
    )


def add_edge(u, v, edge_type):
    graph_edges.append(
        {
            "u_ref": int(u),
            "v_ref": int(v),
            "type": edge_type,

        }
    )


for _, stop_area in stop_areas.iterrows():

    members = stop_area['members']
    all_nodes = _collect_by_roles(members, stop_area['id'])

    for node in chain.from_iterable(all_nodes):
        if "geometry" in node:
            new_lon, new_lat = LineString((xy['lon'], xy['lat']) for xy in node["geometry"]).centroid.xy

            node['lon'], node['lat'] = new_lon[0], new_lat[0]
        if 'lon' not in node:
            node['lon'], node['lat'] = None, None

        add_node(node["ref"], node["lon"], node["lat"], node["role"])

    stations, stops, platforms, entrances, entry_only, exit_only = all_nodes

    for stop in stops:
        for platform in platforms:
            add_edge(stop["ref"], platform["ref"], 'boarding')
            add_edge(platform["ref"], stop["ref"], 'boarding')

    for platform in platforms:
        for station in stations:
            add_edge(station["ref"], platform["ref"], 'subway_station')
            add_edge(platform["ref"], station["ref"], 'subway_station')

    for entrance in entrances:
        for station in stations:
            add_edge(entrance["ref"], station["ref"], 'subway_entrance')
            add_edge(station["ref"], entrance["ref"], 'subway_exit')

    for entrance in entry_only:
        for station in stations:
            add_edge(entrance["ref"], station["ref"], 'subway_entrance')

    for entrance in exit_only:
        for station in stations:
            add_edge(station["ref"], entrance["ref"], 'subway_exit')

for _, stop_area_group in stop_areas_group.iterrows():
    members = stop_area_group['members']

    connect_refs = [station_parent_ref[mem['ref']] for mem in members if mem['ref'] in station_parent_ref]

    pairs = list(combinations(connect_refs, 2))
    for pair in pairs:
        add_edge(pair[0], pair[1], 'subway_transfer')
        add_edge(pair[1], pair[0], 'subway_transfer')
edges_gdf = pd.DataFrame(graph_edges)
nodes_gdf = pd.DataFrame(graph_nodes)

for _, station_data in stations_data.iterrows():
    tags = station_data['tags']
    ref_id = station_data['id']
    station_ind = nodes_gdf[nodes_gdf['ref_id'] == int(ref_id)].index
    nodes_gdf.loc[station_ind, ['depth', 'name']] = (tags.get('depth', 0), tags.get('name', ''))

In [ ]:
nodes_gdf

In [ ]:
nodes_gdf[nodes_gdf['ref_id'] == int(ref_id)].index

In [ ]:
nodes_gdf

In [ ]:
stations_data

In [ ]:
station_parent_ref

In [ ]:
stops

In [ ]:
data[data["is_stop_area_group"]]

In [ ]:
data[data["is_station"]]

In [ ]:
platform_stop_data = data[data["platform_stop_data"]].copy()
platform_stop_data

In [ ]:
def tags(e): return e.get("tags") or {}


elements = data

routes = [e for e in elements if e["type"] == "relation" and tags(e).get("route") == "subway"]
stop_areas = [e for e in elements if e["type"] == "relation" and tags(e).get("public_transport") == "stop_area"]


In [ ]:
pd.DataFrame(stop_areas)

In [ ]:
pd.DataFrame(routes)

In [ ]:
test = routes[0]['members']
for t in

In [ ]:
stop_areas_nodes = {}
for san in stop_areas:
    stop_areas_nodes[san['id']] = {m['ref'] for m in san.get('members', []) if m.get('type') == 'node'}
stop_areas_nodes

In [ ]:
routes_nodes = {}
for rn in routes:
    routes_nodes[rn['id']] = {m['ref'] for m in rn.get('members', []) if m.get('type') == 'node'}

routes_nodes

In [ ]:
pd.DataFrame(stop_areas)

In [ ]:
# Вообщем надо вытаскивать мемберов из routes и stop_areas и маппить как то. Связку делать по нодам stops, которые и там и там есть

In [ ]:
routes

In [ ]:
pd.DataFrame(elements)

In [ ]:
pd.DataFrame(pd.DataFrame(stop_areas).iloc[0]['members'])

In [ ]:
for route_id in routes:


In [ ]:
routes

In [ ]:
route_to_sa

In [ ]:
pd.DataFrame(routes)

In [ ]:
pd.DataFrame(pd.DataFrame(routes).iloc[2]['members'])

In [ ]:
pd.DataFrame(pd.DataFrame(stop_areas).iloc[0]['members'])

In [ ]:
pd.DataFrame(pd.DataFrame(data).iloc[0]['members'])

In [ ]:
from iduedu import get_single_public_transport_graph

subway_graph = get_single_public_transport_graph('subway', osm_id=1114252)

In [ ]:
from iduedu import graph_to_gdf
from iduedu import get_single_public_transport_graph
from iduedu import get_all_public_transport_graph

graph = get_all_public_transport_graph(osm_id=1114252, osm_edge_tags=['charge', 'from', 'name'])
gdf_graph = graph_to_gdf(graph, restore_edge_geom=True, nodes=False)
gdf_graph.explore(column='name')

In [ ]:

from iduedu.enums.network_enums import Network

import pandas as pd
import numpy as np
from shapely.geometry import LineString
import networkx as nx
from shapely import line_merge, MultiLineString
import geopandas as gpd
from iduedu import get_4326_boundary

way_filter = Network.DRIVE.filter
simplify = True
local_crs = 32636

from iduedu.modules.overpass_downloaders import get_network_by_filters
from iduedu import config

config.set_logger_lvl('DEBUG')

bounds = get_4326_boundary(osm_id=337422)
data = get_network_by_filters(bounds, way_filter)

# Разделяем на ноды и линии
nodes = data[data['type'] == 'node'].copy()
ways = data[data['type'] == 'way'].copy()

# Собираем все координаты в лист
coords_list = [
    np.asarray([(p["lon"], p["lat"]) for p in pts], dtype="f8")
    for pts in ways["geometry"].values
]

# определяем начала и концы каждого сегмента
starts = np.concatenate([a[:-1] for a in coords_list], axis=0)
ends = np.concatenate([a[1:] for a in coords_list], axis=0)
M = starts.shape[0]

# определяем длины каждого way для ссылок на индекс
lengths = np.array([a.shape[0] for a in coords_list], dtype=int)
seg_counts = np.maximum(lengths - 1, 0)

# сохраняем ссылки на индексы в ways
way_idx = np.repeat(ways.index.values, seg_counts)

geoms = [LineString([tuple(s), tuple(e)]) for s, e in zip(starts, ends)]

edges = gpd.GeoDataFrame(
    {"way_idx": way_idx},
    geometry=geoms,
    crs=4326
).to_crs(local_crs)
edges = edges.join(ways[["id", "tags"]], on="way_idx")

if simplify:
    simplified_lines = list(line_merge(MultiLineString(edges.geometry.to_list()), directed=True).geoms)
    simplified_lines = gpd.GeoDataFrame(geometry=simplified_lines, crs=local_crs)

    # точки середины линий и sjoin на ближайшую
    line_mid_points = simplified_lines.copy()
    line_mid_points.geometry = line_mid_points.interpolate(simplified_lines.length / 2)

    joined = gpd.sjoin_nearest(line_mid_points[["geometry"]], edges, how="left", max_distance=1)

    joined = joined.reset_index().drop_duplicates(subset="index").set_index("index")

    simplified_lines = simplified_lines.join(joined[['tags']])

    edges = simplified_lines

# Раскрываем необходимые теги
TAG_WHITELIST = ["oneway", "highway", "name", "maxspeed", "lanes"]

tags_data = pd.DataFrame.from_records(
    (
        {k: v for k, v in d.items() if k in TAG_WHITELIST}
        if isinstance(d, dict) else {}
        for d in edges["tags"]
    ),
    index=edges.index
)

edges = edges.join(tags_data)

# определить двусторонние дороги и продублировать перевернув

two_way = edges[edges["oneway"] != "yes"].copy()
two_way.geometry = two_way.geometry.reverse()

edges = pd.concat([edges, two_way], ignore_index=True)

# формируем ноды начала и конца каждой эджи
coords = edges.geometry.get_coordinates().to_numpy()
counts = edges.geometry.count_coordinates()
cuts = np.cumsum(counts)

first_idx = np.r_[0, cuts[:-1]]
last_idx = cuts - 1

starts = coords[first_idx]
ends = coords[last_idx]

edges["start"] = list(map(tuple, starts))
edges["end"] = list(map(tuple, ends))

all_endpoints = pd.Index(edges["start"]).append(pd.Index(edges["end"]))
labels, uniques = pd.factorize(all_endpoints)
n = len(edges)
u = labels[:n]
v = labels[n:]
attrs = edges[TAG_WHITELIST].to_dict("records")

G = nx.DiGraph()

G.add_nodes_from((i, {"x": float(x), "y": float(y)}) for i, (x, y) in enumerate(uniques))
G.add_edges_from(zip(u, v, attrs))

In [ ]:
nx.write_gml(G, "my_graph_simplify.gml", stringizer=lambda x: str(x))

In [ ]:
G.edges(data=True)

In [ ]:
+

In [ ]:
from iduedu.enums.network_enums import Network

import pandas as pd
import numpy as np
from shapely.geometry import LineString
import networkx as nx
from shapely import line_merge, MultiLineString
import geopandas as gpd
from iduedu import get_4326_boundary

way_filter = Network.DRIVE.filter
simplify = True
local_crs = 32636

from iduedu.modules.overpass_downloaders import get_network_by_filters
from iduedu import config

config.set_logger_lvl('DEBUG')

bounds = get_4326_boundary(osm_id=2555133)

In [ ]:
from osmnx import graph_from_polygon

graph_osmnx = graph_from_polygon(bounds, network_type='drive')

In [ ]:
from iduedu import get_drive_graph

iduedu_graph = get_drive_graph(territory=bounds)

In [ ]:
from osmnx import graph_from_polygon

graph_osmnx = graph_from_polygon(bounds, network_type='walk')

In [ ]:
from iduedu import get_walk_graph

iduedu_graph = get_walk_graph(territory=bounds)

In [ ]:
len(graph_osmnx.edges)

In [ ]:
len(iduedu_graph.edges)

In [ ]:
from pyproj import Transformer
from osmnx import project_graph

graph_osmnx = project_graph(graph_osmnx, to_crs=32636)
for node, data in graph_osmnx.nodes(data=True):
    if len(data) != 0:
        graph_osmnx.nodes(data=True)[node]['x'] = float(data['x'])
        graph_osmnx.nodes(data=True)[node]['y'] = float(data['y'])

In [ ]:
nx.write_gml(graph_osmnx, "graph_osmnx_walk.gml", stringizer=lambda x: str(x))

In [ ]:
nx.write_gml(iduedu_graph, "graph_iduedu_walk.gml", stringizer=lambda x: str(x))

In [ ]:

node_ids = set(ways["start"]).union(ways["end"])
needed_nodes = nodes.set_index('id').loc[list(node_ids)].copy()

In [ ]:
from shapely import Point
import geopandas as gpd

needed_nodes['geometry'] = list(map(Point, needed_nodes["lon"], needed_nodes["lat"]))
needed_nodes = gpd.GeoDataFrame(needed_nodes, geometry="geometry", crs=4326)
needed_nodes.explore()

In [ ]:
from networkx import DiGraph

new_graph = DiGraph()



In [ ]:
for _, row in ways.iterrows():



In [ ]:
from networkx import DiGraph

DiGraph().add_edge()

In [ ]:
ways

In [ ]:
gpd.GeoDataFrame(ways, geometry='geometry', crs=4326).explore()

In [ ]:
import geopandas as gpd

gpd.GeoDataFrame(ways, geometry='geometry', crs=4326).explore()

In [ ]:
line_lengths

In [ ]:
ways['geometry'].explode().apply(pd.Series).to_numpy()

In [ ]:
ways['geometry'].apply(len)

In [ ]:
ways['geometry'].apply(pd.Series)

In [ ]:
pd.json_normalize(ways['geometry'].apply(pd.Series))

In [ ]:
nodes[~nodes['tags'].isna()]


In [ ]:
from iduedu import graph_to_gdf

g_gdf = graph_to_gdf(g, restore_edge_geom=True)
g_gdf.explore()

In [ ]:
from shapely import Polygon
import pandas as pd
from iduedu.modules.overpass_downloaders import _poly_to_overpass, _overpass_request
from iduedu import config
from iduedu import get_4326_boundary

poly = get_4326_boundary(osm_id=1114252)


def get_subway_entrances(polygon: Polygon) -> pd.DataFrame:
    polygon_coords = _poly_to_overpass(polygon)
    overpass_query = \
        f"""
        [out:json][timeout:500];
        rel(poly:"{polygon_coords}")["route"="subway"]->.routes;
        node(r.routes)-> .route_nodes;
        rel(bn.route_nodes)->.stop_areas;
        
        rel(br.stop_areas)["public_transport"="stop_area_group"]["type"="public_transport"]->.groups;
        
        nwr(r.stop_areas)["public_transport"="station"]->.stations;
            
        .stop_areas     out geom qt;
        .groups out body qt;
        .stations out tags  qt;
        """

    resp = _overpass_request(
        method="POST",
        overpass_url=config.overpass_url,
        data={"data": overpass_query},
    )
    json_result = resp.json()["elements"]
    if len(json_result) == 0:
        return pd.DataFrame()

    for e in json_result:
        tags = e.get("tags") or {}
        etype = e.get("type")

        e["is_stop_area"] = (etype == "relation" and tags.get("public_transport") == "stop_area")
        e["is_stop_area_group"] = (
                etype == "relation"
                and tags.get("public_transport") == "stop_area_group"
                and tags.get("type") == "public_transport"
        )
        e["is_station"] = (tags.get("public_transport") == "station")

    data = pd.DataFrame(json_result)
    data["transport_type"] = "subway"
    return data


data = get_subway_entrances(poly)

In [ ]:
stop_areas = data[data["is_stop_area"]]
stop_areas_group = data[data["is_stop_area_group"]]
stations_data = data[data["is_station"]]

In [ ]:
from collections import defaultdict
from itertools import chain
from shapely import LineString
from itertools import combinations

graph_nodes = []
graph_edges = []

station_parent_ref = {}


def _collect_by_roles(members, parent_id):
    """Собирает членов по ролям с учётом распространённых синонимов."""
    roles = defaultdict(list)
    for m in (members or []):
        role = (m.get("role") or "").lower()
        roles[role].append(m)

    # базовые наборы
    stations = roles.get("station", [])
    for s in stations:
        station_parent_ref[parent_id] = s['ref']
    stops = roles.get("stop", [])  # stop_position часто идёт как role=stop
    platforms = roles.get("platform", [])

    # входы/выходы и варианты подписей
    entrances = (
            roles.get("entrance", [])
            + roles.get("subway_entrance", [])
            + roles.get("entrance_yes", [])
            + roles.get("entrance_main", [])
    )
    entry_only = (
            roles.get("entry_only", [])
            + roles.get("entrance_entry_only", [])
            + roles.get("entry", [])
    )
    exit_only = (
            roles.get("exit_only", [])
            + roles.get("entrance_exit_only", [])
            + roles.get("exit", [])
    )

    return stations, stops, platforms, entrances, entry_only, exit_only


def add_node(ref_id, x, y, node_type):
    graph_nodes.append(
        {
            "ref_id": int(ref_id),
            "point": (x, y),
            "type": node_type,
        }
    )


def add_edge(u, v, edge_type):
    graph_edges.append(
        {
            "u_ref": int(u),
            "v_ref": int(v),
            "type": edge_type,

        }
    )


for _, stop_area in stop_areas.iterrows():

    members = stop_area['members']
    all_nodes = _collect_by_roles(members, stop_area['id'])

    for node in chain.from_iterable(all_nodes):
        if "geometry" in node:
            new_lon, new_lat = LineString((xy['lon'], xy['lat']) for xy in node["geometry"]).centroid.xy

            node['lon'], node['lat'] = new_lon[0], new_lat[0]
        if 'lon' not in node:
            node['lon'], node['lat'] = None, None

        add_node(node["ref"], node["lon"], node["lat"], node["role"])

    stations, stops, platforms, entrances, entry_only, exit_only = all_nodes

    for stop in stops:
        for platform in platforms:
            add_edge(stop["ref"], platform["ref"], 'boarding')
            add_edge(platform["ref"], stop["ref"], 'boarding')

    for platform in platforms:
        for station in stations:
            add_edge(station["ref"], platform["ref"], 'subway_station')
            add_edge(platform["ref"], station["ref"], 'subway_station')

    for entrance in entrances:
        for station in stations:
            add_edge(entrance["ref"], station["ref"], 'subway_entrance')
            add_edge(station["ref"], entrance["ref"], 'subway_exit')

    for entrance in entry_only:
        for station in stations:
            add_edge(entrance["ref"], station["ref"], 'subway_entrance')

    for entrance in exit_only:
        for station in stations:
            add_edge(station["ref"], entrance["ref"], 'subway_exit')

for _, stop_area_group in stop_areas_group.iterrows():
    members = stop_area_group['members']

    connect_refs = [station_parent_ref[mem['ref']] for mem in members if mem['ref'] in station_parent_ref]

    pairs = list(combinations(connect_refs, 2))
    for pair in pairs:
        add_edge(pair[0], pair[1], 'subway_transfer')
        add_edge(pair[1], pair[0], 'subway_transfer')
edges_gdf = pd.DataFrame(graph_edges)
nodes_gdf = pd.DataFrame(graph_nodes)

for _, station_data in stations_data.iterrows():
    tags = station_data['tags']
    ref_id = station_data['id']
    station_ind = nodes_gdf[nodes_gdf['ref_id'] == int(ref_id)].index
    nodes_gdf.loc[station_ind, ['depth', 'name']] = (tags.get('depth', 0), tags.get('name', ''))

In [ ]:
nodes_gdf

In [ ]:
nodes_gdf[nodes_gdf['ref_id'] == int(ref_id)].index

In [ ]:
nodes_gdf

In [ ]:
stations_data

In [ ]:
station_parent_ref

In [ ]:
stops

In [ ]:
data[data["is_stop_area_group"]]

In [ ]:
data[data["is_station"]]

In [ ]:
platform_stop_data = data[data["platform_stop_data"]].copy()
platform_stop_data

In [ ]:
def tags(e): return e.get("tags") or {}


elements = data

routes = [e for e in elements if e["type"] == "relation" and tags(e).get("route") == "subway"]
stop_areas = [e for e in elements if e["type"] == "relation" and tags(e).get("public_transport") == "stop_area"]


In [ ]:
pd.DataFrame(stop_areas)

In [ ]:
pd.DataFrame(routes)

In [ ]:
test = routes[0]['members']
for t in

In [ ]:
stop_areas_nodes = {}
for san in stop_areas:
    stop_areas_nodes[san['id']] = {m['ref'] for m in san.get('members', []) if m.get('type') == 'node'}
stop_areas_nodes

In [ ]:
routes_nodes = {}
for rn in routes:
    routes_nodes[rn['id']] = {m['ref'] for m in rn.get('members', []) if m.get('type') == 'node'}

routes_nodes

In [ ]:
pd.DataFrame(stop_areas)

In [ ]:
# Вообщем надо вытаскивать мемберов из routes и stop_areas и маппить как то. Связку делать по нодам stops, которые и там и там есть

In [ ]:
routes

In [ ]:
pd.DataFrame(elements)

In [ ]:
pd.DataFrame(pd.DataFrame(stop_areas).iloc[0]['members'])

In [ ]:
for route_id in routes:


In [ ]:
routes

In [ ]:
route_to_sa

In [ ]:
pd.DataFrame(routes)

In [ ]:
pd.DataFrame(pd.DataFrame(routes).iloc[2]['members'])

In [ ]:
pd.DataFrame(pd.DataFrame(stop_areas).iloc[0]['members'])

In [ ]:
pd.DataFrame(pd.DataFrame(data).iloc[0]['members'])

In [ ]:
from iduedu import get_single_public_transport_graph

subway_graph = get_single_public_transport_graph('subway', osm_id=1114252)

In [ ]:
from iduedu import graph_to_gdf
from iduedu import get_single_public_transport_graph
from iduedu import get_all_public_transport_graph

graph = get_all_public_transport_graph(osm_id=1114252, osm_edge_tags=['charge', 'from', 'name'])
gdf_graph = graph_to_gdf(graph, restore_edge_geom=True, nodes=False)
gdf_graph.explore(column='name')

In [ ]:

from iduedu.enums.network_enums import Network

import pandas as pd
import numpy as np
from shapely.geometry import LineString
import networkx as nx
from shapely import line_merge, MultiLineString
import geopandas as gpd
from iduedu import get_4326_boundary

way_filter = Network.DRIVE.filter
simplify = True
local_crs = 32636

from iduedu.modules.overpass_downloaders import get_network_by_filters
from iduedu import config

config.set_logger_lvl('DEBUG')

bounds = get_4326_boundary(osm_id=337422)
data = get_network_by_filters(bounds, way_filter)

# Разделяем на ноды и линии
nodes = data[data['type'] == 'node'].copy()
ways = data[data['type'] == 'way'].copy()

# Собираем все координаты в лист
coords_list = [
    np.asarray([(p["lon"], p["lat"]) for p in pts], dtype="f8")
    for pts in ways["geometry"].values
]

# определяем начала и концы каждого сегмента
starts = np.concatenate([a[:-1] for a in coords_list], axis=0)
ends = np.concatenate([a[1:] for a in coords_list], axis=0)
M = starts.shape[0]

# определяем длины каждого way для ссылок на индекс
lengths = np.array([a.shape[0] for a in coords_list], dtype=int)
seg_counts = np.maximum(lengths - 1, 0)

# сохраняем ссылки на индексы в ways
way_idx = np.repeat(ways.index.values, seg_counts)

geoms = [LineString([tuple(s), tuple(e)]) for s, e in zip(starts, ends)]

edges = gpd.GeoDataFrame(
    {"way_idx": way_idx},
    geometry=geoms,
    crs=4326
).to_crs(local_crs)
edges = edges.join(ways[["id", "tags"]], on="way_idx")

if simplify:
    simplified_lines = list(line_merge(MultiLineString(edges.geometry.to_list()), directed=True).geoms)
    simplified_lines = gpd.GeoDataFrame(geometry=simplified_lines, crs=local_crs)

    # точки середины линий и sjoin на ближайшую
    line_mid_points = simplified_lines.copy()
    line_mid_points.geometry = line_mid_points.interpolate(simplified_lines.length / 2)

    joined = gpd.sjoin_nearest(line_mid_points[["geometry"]], edges, how="left", max_distance=1)

    joined = joined.reset_index().drop_duplicates(subset="index").set_index("index")

    simplified_lines = simplified_lines.join(joined[['tags']])

    edges = simplified_lines

# Раскрываем необходимые теги
TAG_WHITELIST = ["oneway", "highway", "name", "maxspeed", "lanes"]

tags_data = pd.DataFrame.from_records(
    (
        {k: v for k, v in d.items() if k in TAG_WHITELIST}
        if isinstance(d, dict) else {}
        for d in edges["tags"]
    ),
    index=edges.index
)

edges = edges.join(tags_data)

# определить двусторонние дороги и продублировать перевернув

two_way = edges[edges["oneway"] != "yes"].copy()
two_way.geometry = two_way.geometry.reverse()

edges = pd.concat([edges, two_way], ignore_index=True)

# формируем ноды начала и конца каждой эджи
coords = edges.geometry.get_coordinates().to_numpy()
counts = edges.geometry.count_coordinates()
cuts = np.cumsum(counts)

first_idx = np.r_[0, cuts[:-1]]
last_idx = cuts - 1

starts = coords[first_idx]
ends = coords[last_idx]

edges["start"] = list(map(tuple, starts))
edges["end"] = list(map(tuple, ends))

all_endpoints = pd.Index(edges["start"]).append(pd.Index(edges["end"]))
labels, uniques = pd.factorize(all_endpoints)
n = len(edges)
u = labels[:n]
v = labels[n:]
attrs = edges[TAG_WHITELIST].to_dict("records")

G = nx.DiGraph()

G.add_nodes_from((i, {"x": float(x), "y": float(y)}) for i, (x, y) in enumerate(uniques))
G.add_edges_from(zip(u, v, attrs))

In [ ]:
nx.write_gml(G, "my_graph_simplify.gml", stringizer=lambda x: str(x))

In [ ]:
G.edges(data=True)

In [ ]:
+

In [ ]:
from iduedu.enums.network_enums import Network

import pandas as pd
import numpy as np
from shapely.geometry import LineString
import networkx as nx
from shapely import line_merge, MultiLineString
import geopandas as gpd
from iduedu import get_4326_boundary

way_filter = Network.DRIVE.filter
simplify = True
local_crs = 32636

from iduedu.modules.overpass_downloaders import get_network_by_filters
from iduedu import config

config.set_logger_lvl('DEBUG')

bounds = get_4326_boundary(osm_id=2555133)

In [ ]:
from osmnx import graph_from_polygon

graph_osmnx = graph_from_polygon(bounds, network_type='drive')

In [ ]:
from iduedu import get_drive_graph

iduedu_graph = get_drive_graph(territory=bounds)

In [ ]:
from osmnx import graph_from_polygon

graph_osmnx = graph_from_polygon(bounds, network_type='walk')

In [ ]:
from iduedu import get_walk_graph

iduedu_graph = get_walk_graph(territory=bounds)

In [ ]:
len(graph_osmnx.edges)

In [ ]:
len(iduedu_graph.edges)

In [ ]:
from pyproj import Transformer
from osmnx import project_graph

graph_osmnx = project_graph(graph_osmnx, to_crs=32636)
for node, data in graph_osmnx.nodes(data=True):
    if len(data) != 0:
        graph_osmnx.nodes(data=True)[node]['x'] = float(data['x'])
        graph_osmnx.nodes(data=True)[node]['y'] = float(data['y'])

In [ ]:
nx.write_gml(graph_osmnx, "graph_osmnx_walk.gml", stringizer=lambda x: str(x))

In [ ]:
nx.write_gml(iduedu_graph, "graph_iduedu_walk.gml", stringizer=lambda x: str(x))

In [ ]:

node_ids = set(ways["start"]).union(ways["end"])
needed_nodes = nodes.set_index('id').loc[list(node_ids)].copy()

In [ ]:
from shapely import Point
import geopandas as gpd

needed_nodes['geometry'] = list(map(Point, needed_nodes["lon"], needed_nodes["lat"]))
needed_nodes = gpd.GeoDataFrame(needed_nodes, geometry="geometry", crs=4326)
needed_nodes.explore()

In [ ]:
from networkx import DiGraph

new_graph = DiGraph()



In [ ]:
for _, row in ways.iterrows():



In [ ]:
from networkx import DiGraph

DiGraph().add_edge()

In [ ]:
ways

In [ ]:
gpd.GeoDataFrame(ways, geometry='geometry', crs=4326).explore()

In [ ]:
import geopandas as gpd

gpd.GeoDataFrame(ways, geometry='geometry', crs=4326).explore()

In [ ]:
line_lengths

In [ ]:
ways['geometry'].explode().apply(pd.Series).to_numpy()

In [ ]:
ways['geometry'].apply(len)

In [ ]:
ways['geometry'].apply(pd.Series)

In [ ]:
pd.json_normalize(ways['geometry'].apply(pd.Series))

In [ ]:
nodes[~nodes['tags'].isna()]
